**Advanced Python Fundamentals & Functional Programming**

**Metaclasses**

*   **What are Metaclasses?** Metaclasses are "classes of classes."  They control the creation of classes, just like classes control the creation of objects.  By default, the metaclass of a class is `type`.

*   **How Metaclasses Work:** When you define a class, Python calls the metaclass (usually `type`) to create the class object.  You can create custom metaclasses by inheriting from `type` and overriding its `__new__` method (and optionally `__init__`).

*   **Example:**

    ```python
    class MyMetaclass(type):
        def __new__(cls, name, bases, attrs):
            # Modify attributes before class creation
            attrs['attribute_added_by_metaclass'] = True
            print(f"Creating class: {name}")
            print(f"Base Classes : {bases}")
            print(f"Atributtes : {attrs}")
            return super().__new__(cls, name, bases, attrs)

    class MyClass(metaclass=MyMetaclass):
        def my_method(self):
            pass

    obj = MyClass()
    print(obj.attribute_added_by_metaclass)  # Output: True
    ```

    In this example, `MyMetaclass` adds an attribute to `MyClass` during class creation.  The `__new__` method is called *before* the class is created.  It receives the class name, base classes (tuple), and attributes (dictionary).  `super().__new__(...)` calls the base metaclass's `__new__` to actually create the class.

*   **Use Cases (Rare in direct AI, but important for understanding frameworks):**
    *   Enforcing coding standards (e.g., ensuring all classes have a specific method).
    *   Automatically registering classes (e.g., in a plugin system).
    *   Modifying class attributes or methods dynamically.
    *   Creating framework-level features (ORM's, like in Django, use metaclasses).

**Descriptors**

*   **What are Descriptors?** Descriptors are a way to control attribute access. They are Python objects that implement the descriptor protocol: `__get__`, `__set__`, and optionally `__delete__`.

*   **How Descriptors Work:** When you access an attribute on an object, Python checks if the attribute is a descriptor. If it is, Python calls the descriptor's `__get__` method instead of directly returning the attribute's value. Similarly, `__set__` is called for assignment, and `__delete__` for deletion.

*   **Example (Data Descriptor - implements `__set__`):**

    ```python
    class DataValidator:
        def __init__(self, initial_value=None):
            self.value = initial_value

        def __get__(self, instance, owner):
            print(f"Getting value from instance: {instance}, owner: {owner}")
            if instance is None:  # Accessed through the class itself
                return self
            return self.value

        def __set__(self, instance, value):
            print(f"Setting value in instance {instance} to {value}")
            if not isinstance(value, int):
                raise TypeError("Value must be an integer")
            self.value = value

    class MyClass:
        data = DataValidator(10)  # DataValidator instance is the descriptor

    obj = MyClass()
    print(obj.data)  # Access: Calls DataValidator.__get__
    obj.data = 20   # Assignment: Calls DataValidator.__set__
    # obj.data = "hello"  # Raises TypeError

    print(MyClass.data) #Access through the class
    ```

*   **Example (Non-Data Descriptor - only implements `__get__`):**

    ```python
    class NonDataDescriptor:
      def __init__(self, initial_value):
        self.value = initial_value
      def __get__(self, instance, owner):
        print(f"Getting value: {self.value} from instance: {instance}, owner: {owner}")
        if instance is None:
          return self
        return self.value

    class MyClassND:
      attribute = NonDataDescriptor(4)

    my_objectND = MyClassND()
    print(my_objectND.attribute)
    my_objectND.attribute = 8 # Doesn't call __set__
    print(my_objectND.attribute) # Doesn't call __get__
    print(MyClassND.attribute)

    ```

*   **Relationship to Properties and Methods:**  Properties and methods are implemented using descriptors behind the scenes. `property` is a built-in descriptor.

**Advanced Decorators**

*   **Parameterized Decorators:** Decorators that take arguments.

    ```python
    def repeat(num_times):  # Outer function takes arguments
        def decorator_repeat(func):  # Inner function is the actual decorator
            def wrapper(*args, **kwargs):
                for _ in range(num_times):
                    result = func(*args, **kwargs)
                return result
            return wrapper
        return decorator_repeat

    @repeat(num_times=3)  # Apply the parameterized decorator
    def greet(name):
        print(f"Hello, {name}!")

    greet("World")  # Output: Hello, World! (printed 3 times)
    ```

*   **Class-Based Decorators:** Decorators implemented as classes.

    ```python
    class TimeIt:
        def __init__(self, func):
            self.func = func

        def __call__(self, *args, **kwargs):
            import time
            start_time = time.time()
            result = self.func(*args, **kwargs)
            end_time = time.time()
            print(f"Function {self.func.__name__} took {end_time - start_time:.4f} seconds")
            return result

    @TimeIt  # Apply the class-based decorator
    def slow_function(n):
        import time
        time.sleep(n)

    slow_function(2)
    ```
    The `__call__` method makes the class instance callable, like a function.

*   **Use Cases:**
    *   **Memoization (Caching):**  Store the results of expensive function calls to avoid redundant computations.

        ```python
        from functools import lru_cache

        @lru_cache(maxsize=None)  # Built-in memoization decorator
        def fibonacci(n):
            if n < 2:
                return n
            return fibonacci(n-1) + fibonacci(n-2)

        print(fibonacci(10))  # Calculates quickly due to caching
        print(fibonacci.cache_info())
        ```

    *   **Timing Functions:** (See the `TimeIt` example above).
    *   **Argument Validation:** Check the types or values of function arguments.

        ```python
        def validate_types(**types):
            def decorator(func):
                def wrapper(*args, **kwargs):
                    for i, arg in enumerate(args):
                        expected_type = types.get(func.__code__.co_varnames[i])
                        if expected_type and not isinstance(arg, expected_type):
                            raise TypeError(f"Argument {func.__code__.co_varnames[i]} must be of type {expected_type.__name__}")

                    for key, value in kwargs.items():
                      expected_type = types.get(key)
                      if expected_type and not isinstance(value, expected_type):
                        raise TypeError(f"Argument {key} must be of type {expected_type.__name__}")

                    return func(*args, **kwargs)
                return wrapper
            return decorator

        @validate_types(a=int, b=str)  # Specify expected types
        def my_function(a, b, c=None):
            print(f"a: {a}, b: {b}, c: {c}")

        my_function(1, "hello", c=3.14)
        # my_function("hello", 1)  # Raises TypeError
        my_function(1, b = "hi")
        # my_function(1, b = 3) #Raises TypeError
        ```

**Generators and Iterators (Advanced)**

*   **`yield` Keyword:**  Creates a generator function.  When called, a generator function returns a generator object, which is an iterator.  `yield` pauses the function's execution and returns a value.  The next time the generator's `__next__` method is called, execution resumes from where it left off.

*   **Generator Expressions:**  Concise way to create generators, similar to list comprehensions.

    ```python
    # Generator function
    def even_numbers(max_num):
        for n in range(2, max_num + 1, 2):
            yield n

    # Generator expression
    even_numbers_gen = (n for n in range(2, 11, 2))  # Generates even numbers up to 10

    for num in even_numbers(10):
        print(num)

    for num in even_numbers_gen:
        print(num)
    ```

*   **Iterator Protocol:**
    *   `__iter__`:  Returns the iterator object itself (usually `self`).
    *   `__next__`:  Returns the next item in the sequence.  Raises `StopIteration` when there are no more items.

*   **Example (Custom Iterator):**

    ```python
    class MyRange:
        def __init__(self, start, end):
            self.current = start
            self.end = end

        def __iter__(self):
            return self

        def __next__(self):
            if self.current >= self.end:
                raise StopIteration
            else:
                value = self.current
                self.current += 1
                return value

    for i in MyRange(1, 5):
        print(i)
    ```

*   **Difference between `__iter__` and `__next__`:**
    *   An *iterable* object (like a list) has an `__iter__` method that returns an *iterator*.
    *   An *iterator* object has both `__iter__` (which usually just returns itself) and `__next__`.
    *   Generators are iterators, and therefore also iterable.

* **Use in AI**:
    ```python
    def large_dataset_loader(filepath, batch_size):
      with open(filepath, 'r') as file:
        while True:
          batch = []
          for _ in range(batch_size):
            line = file.readline()
            if not line:
              if batch:
                yield batch
              return # StopIteration
            batch.append(line.strip())
          yield batch
    #Assume a large dataset of images in 'image_paths.txt'
    image_loader = large_dataset_loader('image_paths.txt', batch_size=32)
    for image_batch in image_loader:
      #preproces_images(image_batch) -> Assume this function exists
      #train_model(processed_images) -> Assume this function exists
      print(f"Processing batch of size: {len(image_batch)}")
    ```
**Context Managers**

*   **`with` Statement:**  Provides a way to manage resources (like files, locks, network connections) automatically.  Ensures that resources are acquired and released properly, even if exceptions occur.

*   **`__enter__` and `__exit__`:**  Methods that define the context management protocol.
    *   `__enter__`:  Called when entering the `with` block.  Returns a value that can be assigned to a variable using `as`.
    *   `__exit__`:  Called when exiting the `with` block (either normally or due to an exception).  Handles cleanup.

*   **Example (File Handling):**

    ```python
    with open("my_file.txt", "r") as f:  # Automatic file closing
        data = f.read()
    # File is automatically closed here, even if an error occurred
    print(data)
    ```

*   **Example (Custom Context Manager):**

    ```python
    class MyContextManager:
        def __init__(self, resource_name):
            self.resource_name = resource_name
            self.resource = None

        def __enter__(self):
            print(f"Acquiring resource: {self.resource_name}")
            self.resource = f"Resource: {self.resource_name}"  # Simulate acquiring a resource
            return self.resource

        def __exit__(self, exc_type, exc_value, traceback):
            print(f"Releasing resource: {self.resource_name}")
            if exc_type:
                print(f"An exception occurred: {exc_type}, {exc_value}")
            # If __exit__ returns True, the exception is suppressed.
            # If it returns False (or None), the exception is re-raised.
            self.resource = None # Simulate releasing the resource

    with MyContextManager("Database Connection") as db_connection:
        print(f"Using {db_connection}")
        # raise ValueError("Simulated error") # Uncomment to test exception handling

    print("Outside the context")
    ```

**Functional Programming**

*   **`map`, `filter`, `reduce`:**
    *   `map(function, iterable)`: Applies a function to each item in an iterable and returns an iterator of the results.
    *   `filter(function, iterable)`: Filters an iterable, keeping only items for which the function returns `True`.  Returns an iterator.
    *   `reduce(function, iterable)`: Applies a function cumulatively to the items of an iterable, reducing it to a single value.  (From `functools` in Python 3).

*   **Lambda Functions:** Anonymous functions defined using the `lambda` keyword.

    ```python
    from functools import reduce

    numbers = [1, 2, 3, 4, 5]

    # Using map with a lambda function
    squared_numbers = map(lambda x: x**2, numbers)
    print(list(squared_numbers))

    # Using filter with a lambda function
    even_numbers = filter(lambda x: x % 2 == 0, numbers)
    print(list(even_numbers))

    # Using reduce with a lambda function
    product = reduce(lambda x, y: x * y, numbers)
    print(product)
    ```

*   **`functools` and `itertools`:**
    *   `functools`:  Provides higher-order functions (functions that operate on other functions).  Includes `partial` (create functions with some arguments pre-filled), `lru_cache` (memoization).
    *   `itertools`:  Provides functions for creating iterators for efficient looping.  Includes `chain` (combine iterables), `combinations`, `permutations`, `groupby`, and many more.

    ```python
    from functools import partial
    from itertools import chain, combinations

    # functools.partial
    def power(base, exponent):
        return base ** exponent

    square = partial(power, exponent=2)  # Create a new function with exponent fixed
    print(square(5))  # Output: 25

    # itertools.chain
    list1 = [1, 2, 3]
    list2 = [4, 5, 6]
    combined = chain(list1, list2)
    print(list(combined))  # Output: [1, 2, 3, 4, 5, 6]

    # itertools.combinations
    items = ['a', 'b', 'c']
    for combo in combinations(items, 2):
        print(combo)  # Output: ('a', 'b'), ('a', 'c'), ('b', 'c')
    ```
**Concurrency and Parallelism**

*   **Threading (`threading` module):**
    *   **Global Interpreter Lock (GIL):**  In CPython (the standard Python implementation), the GIL allows only one thread to hold control of the Python interpreter at a time.  This means that threads are not truly parallel for CPU-bound tasks.
    *   **Use Cases:** I/O-bound tasks (waiting for network requests, file reads, etc.).  Threads can run concurrently while waiting for I/O.

    ```python
    import threading
    import time

    def worker(name):
        print(f"Thread {name}: Starting")
        time.sleep(2)  # Simulate I/O operation
        print(f"Thread {name}: Finishing")

    threads = []
    for i in range(3):
        t = threading.Thread(target=worker, args=(i,))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()  # Wait for threads to finish

    print("All threads finished")
    ```

*   **Multiprocessing (`multiprocessing` module):**
    *   **Bypassing the GIL:**  Creates separate processes, each with its own Python interpreter and memory space.  Achieves true parallelism for CPU-bound tasks.
    *   **Processes, Pools, Queues, Shared Memory:**
        *   `Process`:  Creates a new process.
        *   `Pool`:  A pool of worker processes that can be used to parallelize tasks.
        *   `Queue`:  A way for processes to communicate with each other by passing messages.
        *   `Value` and `Array`: For sharing simple values or arrays between processes, using shared memory.

    ```python
    import multiprocessing
    import time

    def square(n, results, index):
        time.sleep(1) #Simulate workload
        results[index] = n * n

    if __name__ == "__main__":  # Important for multiprocessing
        numbers = [1, 2, 3, 4, 5]
        # Using Array for shared memory. 'i' means integer type
        results = multiprocessing.Array('i', len(numbers))

        processes = []
        for i, num in enumerate(numbers):
            p = multiprocessing.Process(target=square, args=(num, results, i))
            processes.append(p)
            p.start()

        for p in processes:
            p.join()

        print("Results (using shared Array):", list(results))

        # --- Using Pool and map ---
        def cube(n):
          time.sleep(1)
          return n*n*n

        with multiprocessing.Pool(processes=4) as pool:
          cubed_numbers = pool.map(cube, numbers)

        print("Results using Pool", cubed_numbers)

        # --- Using Queue ---
        def producer(queue, data):
          for item in data:
            print(f"Producing {item}")
            time.sleep(0.5)
            queue.put(item)

        def consumer(queue):
          while True:
            item = queue.get()
            if item is None:
              break
            print(f"Consuming {item}")
            time.sleep(1)

        q = multiprocessing.Queue()
        data_to_process = [1,2,3,4, None] #None as sentinel value
        p1 = multiprocessing.Process(target=producer, args=(q, data_to_process))
        p2 = multiprocessing.Process(target=consumer, args=(q,))

        p1.start()
        p2.start()

        p1.join()
        p2.join()

    ```

*   **Asynchronous Programming (`asyncio`):**
    *   `async` and `await`:  Keywords for defining and awaiting asynchronous operations.
    *   **Event Loop:**  Manages the execution of asynchronous tasks.
    *   **Use Cases:**  High-concurrency I/O-bound tasks (e.g., handling many network connections).

    ```python
    import asyncio
    import time

    async def my_coroutine(name, delay):
        print(f"Coroutine {name}: Starting")
        await asyncio.sleep(delay)  # Simulate an asynchronous I/O operation
        print(f"Coroutine {name}: Finishing")
        return f"Result from {name}"

    async def main():
        # Create tasks
        task1 = asyncio.create_task(my_coroutine("A", 2))
        task2 = asyncio.create_task(my_coroutine("B", 1))

        # Run tasks concurrently
        results = await asyncio.gather(task1, task2)  # Await multiple tasks
        print(results)
        print(f"Main Finishing")


    if __name__ == "__main__":
      start_time = time.time()
      asyncio.run(main())  # Run the event loop
      end_time = time.time()
      print(f"Total execution time: {end_time-start_time:.4f}s")
    ```

This course provides a comprehensive overview of the requested advanced Python topics. Remember that practice is key to mastering these concepts. Work through the examples, modify them, and try applying them to your own projects. Good luck!
